In [10]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras.layers import *
import tensorflow
import math
import h5py
from os import path

In [ ]:
DATASET_DIR      = '../dataset'
dataset          = h5py.File(path.join(DATASET_DIR, 'dataset.hdf5'))

test_images      = dataset['images/test']
test_labels      = dataset['labels/test']

train_images     = dataset['images/train']
train_labels     = dataset['labels/train']

pca_eigenvectors = dataset['pca/eigenvectors']
pca_mean         = dataset['pca/mean']

In [11]:
def euclidean(y_true, y_pred):
    n = int(y_pred.get_shape()[1])
    return tensorflow.reduce_sum((y_true - y_pred) ** 2) / (2 * n)

In [12]:
# Caffe has a learning rate decay policy called 'inv'.
# The learning rate equals base_lr * (1 + gamma * epoch) ^ (-power).
def inv_decay(base_lr, gamma, power):
    def decay(epoch):
        return base_lr * (1 + gamma * epoch) ** (-power)
    
    return decay

In [13]:
# Caffe's Xavier filler uses a uniform distribution with a
# standard deviation of sqrt(3 / fan_in) by default.
# This is almost, but not quite equivalent to Keras' he_uniform initializer,
# which uses a standard deviation of sqrt(6 / fan_in).
def xavier(shape, name=None, dim_ordering='th'):
    return initializations.he_uniform(shape, name, dim_ordering) / math.sqrt(2)

In [14]:
def gaussian(std_dev):
    def init(shape, name=None):
        return initializations.normal(shape, std_dev, name)
    
    return init

In [15]:
model = Sequential([
        Convolution2D(
            nb_filter   = 8,
            nb_row      = 5,
            nb_col      = 5,
            init        = xavier,
            input_shape = (128, 128, 1)
        ),
        MaxPooling2D(
            pool_size   = (2, 2)
        ),
        LeakyReLU(
            alpha       = 0.05
        ),
        Convolution2D(
            nb_filter   = 8,
            nb_row      = 5,
            nb_col      = 5,
            init        = xavier
        ),
        MaxPooling2D(
            pool_size   = (2, 2)
        ),
        LeakyReLU(
            alpha       = 0.05
        ),
        Convolution2D(
            nb_filter   = 8,
            nb_row      = 5,
            nb_col      = 5,
            init        = xavier
        ),
        LeakyReLU(
            alpha       = 0.05
        ),
        Flatten(),
        Dense(
            output_dim  = 1024,
            init        = gaussian(std_dev=0.01),
            activation  = 'relu'
        ),
        Dense(
            output_dim  = 1024,
            init        = gaussian(std_dev=0.05),
            activation  = 'relu'
        ),
        Dense(
            output_dim  = 22,
            init        = gaussian(std_dev=0.02)
        ),
        Dense(
            output_dim  = 28,
            weights     = (pca_eigenvectors, pca_mean),
            trainable   = False
        )
    ])

In [16]:
model.compile(
    optimizer = SGD(
        lr       = 0.000005,
        momentum = 0.9
    ),
    loss         = euclidean
)

In [9]:
model.fit(
    train_images,
    train_labels,
    batch_size = 64,
    nb_epoch   = 40000,
    callbacks  = [LearningRateScheduler(
            inv_decay(
                base_lr = 0.000005,
                gamma   = 0.0001,
                power   = 0.75
            ))])

NameError: name 'train_images' is not defined

In [ ]:
model.evaluate(
    test_images,
    test_labels,
    batch_size = 64
)